In [718]:
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

In [698]:
verdadero = pd.read_csv('True.csv')

In [699]:
falso = pd.read_csv('Fake.csv')

In [700]:
verdadero['label'] = 1
falso['label'] =  0

In [701]:
verdadero.head()

,title,text,subject,date,label
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",1
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",1
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",1
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",1
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",1


In [702]:
data = pd.concat([verdadero,falso], axis=0)
data.head()


,title,text,subject,date,label
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",1
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",1
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",1
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",1
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",1


In [703]:
data = data.drop(columns=['title','subject','date'])

In [704]:
data = data.sample(frac = 1) #Mezclamos los datos 
data.head()

,text,label
21144,MEXICO CITY (Reuters) - Trade negotiators from...,1
5011,It s getting to the point that as frightening ...,0
6841,There was once a time that John Kasich seemed ...,0
8990,"Thus far, the Oregon militia members who have ...",0
12031,"In this chess match of political wills, Putin ...",0


In [705]:
data.reset_index(inplace=True)

In [706]:
data.head()

,index,text,label
0,21144,MEXICO CITY (Reuters) - Trade negotiators from...,1
1,5011,It s getting to the point that as frightening ...,0
2,6841,There was once a time that John Kasich seemed ...,0
3,8990,"Thus far, the Oregon militia members who have ...",0
4,12031,"In this chess match of political wills, Putin ...",0


In [707]:
data.drop(columns=['index'], axis = 1, inplace = True)

In [708]:
data.isnull().sum()

text     0
label    0
dtype: int64

In [709]:
def correction(text):
    text = text.lower()
    text = re.sub(r'\[.*?\]', '', text)

    #Remover URL's
    text = re.sub(r'https?://\S+\www\.\S+', '', text)
    
    #Remover HTML tags
    text = re.sub(r'<.*?>', '', text)

    #Remover punctuation
    text = re.sub(r'[^\w\s]', '', text)
    
    text = re.sub(r'[%s]' %re.escape(string.punctuation),'',text)

    text = re.sub(r'\w*\d\w*', '', text)

    # Remover saltos de linea
    text = re.sub(r'\n', '', text)

    #Remover digitos
    text = re.sub(r'\d', '', text)

    return text

In [710]:
data['text'] = data['text'].apply(correction)

In [711]:
X, y = data['text'], data['label']

In [719]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [720]:
# TF-IDF vectorization
vectorization = TfidfVectorizer(stop_words='english', max_df=0.7)
xv_train = vectorization.fit_transform(X_train)
xv_test = vectorization.transform(X_test)

In [721]:
lr = LogisticRegression()
lr.fit(xv_train, y_train)

LogisticRegression()

In [722]:
lr.score(xv_test, y_test)

0.9859688195991091

In [723]:
prediction = lr.predict(xv_test)

In [724]:
print(classification_report(y_test, prediction))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      4643
           1       0.98      0.99      0.99      4337

    accuracy                           0.99      8980
   macro avg       0.99      0.99      0.99      8980
weighted avg       0.99      0.99      0.99      8980



In [725]:
svc = LinearSVC()
svc.fit(xv_train, y_train)

/home/scavenger/miniconda3/envs/py312/lib/python3.10/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(


LinearSVC()

In [729]:
svc.score(xv_test, y_test)

0.9943207126948775

In [731]:
svc_prediction = svc.predict(xv_test)
matriz = confusion_matrix(y_test,svc_prediction)
print('Matriz de Confusion')
print(matriz)

Matriz de Confusion
[[4612   31]
 [  20 4317]]


In [728]:
print(classification_report(y_test, svc_prediction))

              precision    recall  f1-score   support

           0       1.00      0.99      0.99      4643
           1       0.99      1.00      0.99      4337

    accuracy                           0.99      8980
   macro avg       0.99      0.99      0.99      8980
weighted avg       0.99      0.99      0.99      8980

